# Julia version of bonding models for PES !!

### Basic Optimization tests

In [88]:
using Optim

In [94]:
# standard opt:
rosenbrock(x) =  (1.0 - x[1])^2 + 99.0 * (x[2] - x[1]^2)^2
result = optimize(rosenbrock, [1.1, 2.3], BFGS())

 * Status: success

 * Candidate solution
    Final objective value:     5.264694e-17

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 9.47e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.47e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.53e-15 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 6.71e+01 ≰ 0.0e+00
    |g(x)|                 = 3.92e-10 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    15
    f(x) calls:    46
    ∇f(x) calls:   46


In [104]:
# examples of "closures", a technique to fix args!
f(x::Vector, a, b) = (x[1] - a)^2 + (x[2] - b)^2

using Optim
g(x::Vector) = f(x, 3, 4)
optimize(g, [0.,0.])

 * Status: success

 * Candidate solution
    Final objective value:     1.226133e-10

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    50
    f(x) calls:    100


In [106]:
# another closure example in Julia, anonymous function:
optimize(x -> f(x,3,4), [0.,0.]) 

 * Status: success

 * Candidate solution
    Final objective value:     1.226133e-10

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    50
    f(x) calls:    100


In [1]:
# functional form:
function f_ratpot_2(Θ, R, M)
    #=
    ansatz 1 for diatomic potential
    params:
        - Θ := training parameters, vector ()
        - R := distances, vector
    =#
    # unroll coefficients
    a = Θ[1:M]
    b = Θ[M+1:2*M]
    c = Θ[2*M+1:3*M+4]
    d = Θ[3*M+5:4*M+7]
    
    # b_i ≥ 0 for i > 1:
    t = b[2:M]
    bool = t .< 0.
    t[bool] = -t[bool]
    b[2:M] = t
    
    # d_i ≥ 0 for i > 0:
    bool = d .< 0.
    d[bool] = -d[bool]    
    println(b)
    println(d)
    
    # evaluate P:
    P = c[end-1]
    P = P .* ((R .- a[1]).^2 .+ (b[1] .* R))
    for i=2:M
        P .*= (R .- a[i]).^2 .+ (b[i]*R)
    end

    
    # eval Q:
    Q = (R .+ d[end]).*R
    for i=1:M+2
        Q .*= (R .- c[i]).^2 .+ d[i].*R
    end
    
    # eval potential:
    V = c[end] .+ (P ./ Q)
    return V
end

f_ratpot_2 (generic function with 1 method)

In [2]:
M = 2
Θ = collect(LinRange(-1., -2., 21))
Θ = Θ[1:4*M+7] # 4M+7 params
println(Θ, size(Θ))
R = collect(LinRange(0., 5., 6)) .+ 1
println(R)
f_ratpot_2(Θ, R, M)

[-1.0, -1.05, -1.1, -1.15, -1.2000000000000002, -1.25, -1.2999999999999998, -1.35, -1.4, -1.4500000000000002, -1.5, -1.55, -1.6, -1.65, -1.7](15,)
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
[-1.1, 1.15]
[1.5, 1.55, 1.6, 1.65, 1.7]


6-element Vector{Float64}:
 -1.4538848497222299
 -1.4504032610414908
 -1.450089540167252
 -1.4500280979713995
 -1.450010836561844
 -1.4500048065726545

### 8.1 Bonding features

In [95]:
function t_R_fun(R, R_up, R_low, e)
    R2 = R.^2
    return ((R2 .- R_low^2)./(R_up^2 .- R2)).^e
end

#function s_bond_strength

t_R_fun (generic function with 3 methods)

In [97]:
R = [2.5, 3.1, 5, 1.1]
@time t_R_fun(R, 6, 1, 2)

  0.000006 seconds (2 allocations: 224 bytes)


4-element Vector{Float64}:
 0.031141868512110732
 0.1064455529835572
 4.760330578512396
 3.6435919338971527e-5